### Model and Prompt

`langchain`은 50개 이상의 서드파티 공급업체 및 플랫폼과의 통합을 제공하며, OpenAI, Azure OpenAI, Databricks, MosaicML은 물론 허깅페이스 허브 및 오픈소스 LLM 세계와의 통합도 지원한다. 

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [8]:
from langchain_openai import OpenAI
llm = OpenAI(
    openai_api_key = openai.api_key, 
    max_tokens = 1024
)

In [10]:
print(llm.invoke('2025년 가격이 상승할 것으로 기대되는 미국 증시의 섹터나 테마는 뭘까? 3개만 알려줘'))



1. 기술 섹터 (Technology Sector): 기술 기업들은 빠르게 발전하고 있으며, 인공지능, 빅데이터, 클라우드 컴퓨팅 등의 기술을 통해 더 많은 성장 가능성을 가지고 있습니다. 또한, 코로나19 이후 원격 근무와 온라인 비즈니스의 증가로 인해 기술 기업들의 수요가 더욱 증가할 것으로 예상됩니다.

2. 건강 관리 섹터 (Healthcare Sector): 인구 고령화와 건강식품 등 건강 관리에 대한 관심이 높아지면서 건강 관리 기업들의 성장이 기대됩니다. 또한, 코로나19 이후 의료 기술의 발전과 백신 개발 등에 대한 수요가 더욱 증가할 것으로 예상됩니다.

3. 친환경 에너지 섹터 (Renewable Energy Sector): 기후 변화에 대한 전 세계적인 관심이 높아지면서 친환경 에너지 기업들의 성장이 가속화될 것으로 예상됩니다. 특히 미국 정부의 친환경 에너지 정책의 변화로 인해 더욱 많은 투자가 이루어질 것으로 예상됩니다.


#### prompt template

prompt template은 언어 모델에 대한 프롬프트를 생성하는 방법을 정의하는 구성 요소이다. 여기에는 변수, placeholder, prefix, suffix, 그리고 데이터와 과업에 따라 맞춤화할 수 있는 기타 요소가 포함될 수 있다.
예를 들어, 언어 모델을 사용하여 한 언어에서 다른 언어로 번역을 생성하고 싶다고 가정해 보자. 다음과 같은 prompt template을 사용할 수 있다.

```
sentence : {sentence}
translate to {language} :
```

이 템플릿은 다음과 같이 쉽게 구현할 수 있다.

In [11]:
from langchain import PromptTemplate

template = """
문장 : {sentence}
{language}로 번역 :
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ['sentence','language']
)

In [12]:
print(
    prompt.format(
        sentence = "탁자 위에 고양이가 있다.",
        language = '영어'
    )
)


문장 : 탁자 위에 고양이가 있다.
영어로 번역 :



- completion model : 텍스트 입력을 받아 텍스트 출력을 생성하는 LLM의 한 유형이다. 과업과 훈련한 데이터에 따라 일관되고 관련성 있는 방식으로 prompt를 이어가려고 노력한다. 예를 들어, 완성 모델을 프롬프트에 따라서 요약, 번역, 스토리, 코드, 가사 등을 생성할 수 있다.
- chat model : 대화 응답을 생성하도록 설계된 특수한 종류의 완성 모델이다. 메세지 목록을 입력으로 받으며, 각 메세지에는 역할(시스템, 사용자 또는 어시스턴트)과 콘텐츠가 있다. 채팅 모델은 이전 메세지와 시스템 지시를 기반으로 어시스턴트 역할에 대한 새 메세지를 생성하려고 시도한다.

완성모델과 채팅 모델의 주요 차이점은 완성 모델을 하나의 텍스트 입력을 프롬프트로 기대하는 반면, 채팅 모델은 메세지 목록을 입력으로 기대한다는 점이다.